In [1]:
# given a search query, return all results which are related that query

In [1]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

from vectorize_dataset import load_descriptions_data, create_db

In [2]:
hf_df = load_descriptions_data()
db = create_db(hf_df)

/Users/noahkasmanoff/anaconda3/envs/nlp/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Found cached dataset parquet (/Users/noahkasmanoff/.cache/huggingface/datasets/nkasmanoff___parquet/nkasmanoff--huggingface-datasets-60bbbd3d2e18598e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 82.36it/s]
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1140, whic

In [3]:




# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)


In [17]:



# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)


query = "Show me datasets which tell me about natural disasters?"
result = qa({"query": query})
reponse_text = result['result']
source_documents = result['source_documents']
linked_datasets = [x.metadata['id'] for x in source_documents]

print("Answer: ", reponse_text)
print("Related Datasets: ", linked_datasets)


Answer:   This dataset contains 30,000 messages drawn from events including an earthquake in Haiti in 2010, an earthquake in Chile in 2010, floods in Pakistan in 2010, super-storm Sandy in the U.S.A. in 2012, and news articles spanning a large number of years and 100s of different disasters. The HumAID Twitter dataset consists of several thousands of manually annotated tweets that has been collected during 19 major natural disaster events including earthquakes, hurricanes, wildfires, and floods, which happened from 2016 to 2019 across different parts of the World.
Related Datasets:  ['disaster_response_messages', 'Firoj/HumAID']


['SharedBailii/NER-BAILII-UK-CCA', 'svnfs/depth-of-field']